In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from joblib import load
import os
import re
from sklearn.metrics import mean_absolute_percentage_error

## IMPORTAR MODELOS DE REGRESION ENTRENADOS

In [4]:
# average_cluster_0=pd.read_csv("../data/interim/series_cluster_0.csv",parse_dates=["Date/Time"],index_col="Date/Time")
# average_cluster_1=pd.read_csv("../data/interim/series_cluster_1.csv",parse_dates=["Date/Time"],index_col="Date/Time")
# average_cluster_2=pd.read_csv("../data/interim/series_cluster_2.csv",parse_dates=["Date/Time"],index_col="Date/Time")
# average_cluster_3=pd.read_csv("../data/interim/series_cluster_3.csv",parse_dates=["Date/Time"],index_col="Date/Time")

In [5]:
# closest_cluster_0=pd.read_csv("../data/interim/without_closest_cluster_0.csv",parse_dates=["Date/Time"],index_col="Date/Time")
# closest_cluster_1=pd.read_csv("../data/interim/without_closest_cluster_1.csv",parse_dates=["Date/Time"],index_col="Date/Time")
# closest_cluster_2=pd.read_csv("../data/interim/without_closest_cluster_2.csv",parse_dates=["Date/Time"],index_col="Date/Time")
# closest_cluster_3=pd.read_csv("../data/interim/without_closest_cluster_3.csv",parse_dates=["Date/Time"],index_col="Date/Time")

In [6]:
modelo_average_cluster_0= load('../models/modelo_average_cluster_0.joblib')
scaler_average_cluster_0= load('../models/scaler_average_cluster_0.joblib')
modelo_average_cluster_1= load('../models/modelo_average_cluster_1.joblib')
scaler_average_cluster_1= load('../models/scaler_average_cluster_1.joblib')
modelo_average_cluster_2= load('../models/modelo_average_cluster_2.joblib')
scaler_average_cluster_2= load('../models/scaler_average_cluster_2.joblib')
modelo_average_cluster_3= load('../models/modelo_average_cluster_3.joblib')
scaler_average_cluster_3= load('../models/scaler_average_cluster_3.joblib')

In [7]:
modelo_closest_cluster_0= load('../models/modelo_closest_cluster_0.joblib')
scaler_closest_cluster_0= load('../models/scaler_closest_cluster_0.joblib')
modelo_closest_cluster_1= load('../models/modelo_closest_cluster_1.joblib')
scaler_closest_cluster_1= load('../models/scaler_closest_cluster_1.joblib')
modelo_closest_cluster_2= load('../models/modelo_closest_cluster_2.joblib')
scaler_closest_cluster_2= load('../models/scaler_closest_cluster_2.joblib')
modelo_closest_cluster_3= load('../models/modelo_closest_cluster_3.joblib')
scaler_closest_cluster_3= load('../models/scaler_closest_cluster_3.joblib')

In [8]:
def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

## IMPORTAR DATOS PARA LAS PRUEBAS

In [9]:
# Especifica el directorio donde están los archivos CSV
carpeta ="../data/raw/"
# Obtén la lista de todos los archivos en la carpeta
archivos = os.listdir(carpeta)
# Define the columns to keep
columns_to_keep = [
    'Date/Time',
    'Electricity:Facility [kW](Hourly)',
    'Fans:Electricity [kW](Hourly)',
    'Cooling:Electricity [kW](Hourly)',
    'Heating:Electricity [kW](Hourly)',
    'InteriorLights:Electricity [kW](Hourly)',
    'InteriorEquipment:Electricity [kW](Hourly)'
]
# Load and concatenate data from all files
dfs = []
for file in archivos:
    df = pd.read_csv(f"../data/raw/{file}", usecols=columns_to_keep)
    df["Date/Time"] = '2004 ' + df["Date/Time"]
    date_format = '%Y %m/%d %H:%M:%S'
    df["Date/Time"] = pd.to_datetime(df["Date/Time"], format=date_format, errors='coerce')
    match = re.match(r'^[^_]+', file)
    name = match.group(0)
    df["type_building"] = name
    dfs.append(df)

In [10]:
#se extraen los datos clusterizados
data_clustered=pd.read_csv("../data/interim/clustering_pca.csv")

In [11]:
#se cargan los datos mas cercanos a los clusters
closest_points_data=pd.read_csv("../data/interim/closest_points.csv")
closest_points_data=closest_points_data.rename(columns={"Unnamed: 0":"indice_closest"})
closest_points_data

,indice_closest,PC1,PC2,PC3,Cluster_KMeans,Cluster_DBSCAN,Cluster_Hierarchical
0,67,-1.122322,-0.026253,0.075707,0,10,0
1,15,4.515147,1.414719,-1.786065,1,-1,1
2,5,5.561883,-2.633525,0.505042,2,-1,2
3,45,2.758477,2.785423,1.366780,1,-1,3


In [12]:
#OBTENER LAS SERIES MAS CERCANAS PARA CADA CLUSTER
indices_clusters=closest_points_data.indice_closest.unique().tolist()
closest_series=[]
for indice in indices_clusters:
    closest_series.append(dfs[indice])

### OBTENER DATOS DE PRUEBA PARA LOS MODELOS ENTRENADOS CON LAS SERIE PROMEDIO DE CADA CLUSTER 

In [13]:
# Agrupar dataframes por cluster
clusters = data_clustered['Cluster_Hierarchical'].unique()
average_dfs = {cluster: [] for cluster in clusters}

for i, df in enumerate(dfs):
    cluster = data_clustered.loc[i, 'Cluster_Hierarchical']
    average_dfs[cluster].append(df)

### OBTENER DATOS DE PRUEBA PARA LOS MODELOS ENTRENADOS CON LAS SERIE MAS CERCANA DE CADA CLUSTER 

In [14]:
indices_clusters = closest_points_data.indice_closest.unique().tolist()

# Filtrar la lista de DataFrames eliminando aquellos en los índices de indices_clusters
dfs_actualizados = [df for i, df in enumerate(dfs) if i not in indices_clusters]

In [15]:
# Agrupar dataframes por cluster
clusters = data_clustered['Cluster_Hierarchical'].unique()
closest_dfs = {cluster: [] for cluster in clusters}

for i, df in enumerate(dfs_actualizados):
    cluster = data_clustered.loc[i, 'Cluster_Hierarchical']
    closest_dfs[cluster].append(df)

# PRUEBAS CLUSTER 0

## closest

In [50]:
# Inicializar variables para acumular resultados
rmse_list = []
smape_list = []
# Lista de columnas para análisis
columnas = ['Fans:Electricity [kW](Hourly)', 'Cooling:Electricity [kW](Hourly)',
       'InteriorLights:Electricity [kW](Hourly)',
       'InteriorEquipment:Electricity [kW](Hourly)', 'hour', 'day_of_week',
       'month']

for df_serie in closest_dfs[0]:
    df=df_serie.copy()
    df["Date/Time"]=pd.to_datetime(df["Date/Time"])
    df.set_index("Date/Time",inplace=True)
    # Extraer características temporales
    df['hour'] = df.index.hour
    df['day_of_week'] = df.index.dayofweek
    df['month'] = df.index.month
    
    X = df.loc[:, columnas].copy()
    y = df.loc[:, 'Electricity:Facility [kW](Hourly)'].copy()
    
    # Escalar las características
    X_scaled = scaler_closest_cluster_0.transform(X)
    
    # Predecir y calcular métricas de error
    y_pred = modelo_closest_cluster_0.predict(X_scaled)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    smape_value = smape(y, y_pred)  # Calcular sMAPE
    
    # Almacenar resultados
    rmse_list.append(rmse)
    smape_list.append(smape_value)

# Calcular promedios de las métricas de error
average_rmse = np.mean(rmse_list)
average_smape = np.mean(smape_list)

# Resultados
results = {
    "Metricas del modelo": {
        'Average RMSE': average_rmse,
        'Average sMAPE (%)': average_smape
    }
}


In [51]:
# Mostrar los resultados
print("Resultados del modelo:")
optimized_rf_results_df=pd.DataFrame(results)
optimized_rf_results_df

Resultados del modelo:


,Metricas del modelo
Average RMSE,76.949376
Average sMAPE (%),52.271431


In [52]:
rmse_list

[6.124977211781756,
 6.1278036919600725,
 6.197875105992481,
 6.026074754707554,
 6.025296315720119,
 203.70755366176908,
 205.00498602915061,
 194.4453555257644,
 194.97655537691267,
 830.6193049863855,
 54.69345000469319,
 61.693343187307306,
 58.79954007173386,
 4.830397025863422,
 4.8093800977265095,
 4.814209001054319,
 4.796775708364841,
 4.81574280283526,
 108.88649989163953,
 106.42613645929026,
 107.93942627931271,
 108.23492192095583,
 106.62639512881027,
 70.84029433481471,
 71.61286768329569,
 73.32792465189345,
 66.43049626922893,
 66.77146102112283,
 5.796299815774096,
 5.76197361871275,
 5.744187582670508,
 5.855106404705105,
 5.834105355981151,
 457.1233357849169,
 473.6878822321656,
 22.04843267954875,
 20.694731129053864,
 10.86641495139946,
 10.83659267562603,
 10.817813108599374,
 10.943746129395652,
 10.934001678318582,
 5.439815304191012,
 5.423381679651442,
 5.400858204763316,
 5.397140137813658,
 5.4127557749034505,
 0.6777456211589287,
 0.719578159637776,
 0.99

## Average

In [53]:
# Inicializar variables para acumular resultados
rmse_list = []
smape_list = []

In [54]:
# Lista de columnas para análisis
columnas = ['Fans:Electricity [kW](Hourly)', 'Cooling:Electricity [kW](Hourly)',
            'InteriorLights:Electricity [kW](Hourly)',
            'InteriorEquipment:Electricity [kW](Hourly)', 'hour', 'day_of_week',
            'month']

for df_serie in average_dfs[0]:
    df = df_serie.copy()
    df["Date/Time"] = pd.to_datetime(df["Date/Time"])
    df.set_index("Date/Time", inplace=True)
    # Extraer características temporales
    df['hour'] = df.index.hour
    df['day_of_week'] = df.index.dayofweek
    df['month'] = df.index.month
    df.dropna(inplace=True)
    X = df.loc[:, columnas].copy()
    y = df.loc[:, 'Electricity:Facility [kW](Hourly)'].copy()
    
    # Escalar las características
    X_scaled = scaler_average_cluster_0.transform(X)
    
    # Predecir y calcular métricas de error
    y_pred = modelo_average_cluster_0.predict(X_scaled)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    smape_value = smape(y, y_pred)  # Calcular sMAPE
    
    # Almacenar resultados
    rmse_list.append(rmse)
    smape_list.append(smape_value)

# Calcular promedios de las métricas de error
average_rmse = np.mean(rmse_list)
average_smape = np.mean(smape_list)

# Resultados
results = {
    "Metricas del modelo": {
        'Average RMSE': average_rmse,
        'Average sMAPE (%)': average_smape
    }
}


In [55]:
# Mostrar los resultados
print("Resultados del modelo:")
optimized_rf_results_df=pd.DataFrame(results)
optimized_rf_results_df

Resultados del modelo:


,Metricas del modelo
Average RMSE,51.425706
Average sMAPE (%),62.853891


In [45]:
rmse_list

[18.941344435982266,
 18.775233435252886,
 18.557105212936975,
 19.367520143142656,
 19.30393893298891,
 159.82915470171372,
 162.10058933321923,
 163.17978046029444,
 153.6936651551147,
 154.11754575720062,
 44.374161710342,
 42.316849955025916,
 43.13268718207002,
 45.678872828477346,
 42.45138932729322,
 24.295682774051315,
 24.113274181575967,
 23.839092174970588,
 24.658909574593874,
 24.647738065018213,
 71.7950211253372,
 70.10874162565067,
 71.58295231577608,
 70.50571324541474,
 69.39246436719984,
 53.02309662124517,
 54.39915524770963,
 56.50079585522488,
 49.22448582025587,
 48.71672801988252,
 26.70209318829037,
 26.581120748631644,
 26.414541234125696,
 27.0253999552208,
 26.99645804815017,
 16.708115574899185,
 16.544546293104375,
 16.32013237895723,
 16.35645361945462,
 17.058281532364916,
 31.955375413000034,
 31.925794164013823,
 31.864470764356774,
 32.05267411345526,
 32.06676477388521,
 44.18531243330538,
 44.061002300555685,
 43.7123654688419,
 44.993020885547956,


# PRUEBAS CLUSTER 1

## closest

In [46]:
# Lista de columnas para análisis
columnas = ['Fans:Electricity [kW](Hourly)', 'Cooling:Electricity [kW](Hourly)',
       'InteriorLights:Electricity [kW](Hourly)',
       'InteriorEquipment:Electricity [kW](Hourly)', 'hour', 'day_of_week',
       'month']
# Inicializar variables para acumular resultados
rmse_list = []
smape_list = []
for df_serie in closest_dfs[1]:
    df=df_serie.copy()
    df["Date/Time"]=pd.to_datetime(df["Date/Time"])
    df.set_index("Date/Time",inplace=True)
    # Extraer características temporales
    df['hour'] = df.index.hour
    df['day_of_week'] = df.index.dayofweek
    df['month'] = df.index.month
    
    X = df.loc[:, columnas].copy()
    y = df.loc[:, 'Electricity:Facility [kW](Hourly)'].copy()
    
    # Escalar las características
    X_scaled = scaler_closest_cluster_1.transform(X)
    
    # Predecir y calcular métricas de error
    y_pred = modelo_closest_cluster_1.predict(X_scaled)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    smape_value = smape(y, y_pred)  # Calcular sMAPE
    
    # Almacenar resultados
    rmse_list.append(rmse)
    smape_list.append(smape_value)

# Calcular promedios de las métricas de error
average_rmse = np.mean(rmse_list)
average_smape = np.mean(smape_list)

# Resultados
results = {
    "Metricas del modelo": {
        'Average RMSE': average_rmse,
        'Average sMAPE (%)': average_smape
    }
}

In [47]:
# Mostrar los resultados
print("Resultados del modelo")
optimized_rf_results_df=pd.DataFrame(results)
optimized_rf_results_df

Resultados del modelo


,Metricas del modelo
Average RMSE,101.841944
Average sMAPE (%),49.170538


## Average

In [48]:
# Lista de columnas para análisis
columnas = ['Fans:Electricity [kW](Hourly)', 'Cooling:Electricity [kW](Hourly)',
       'InteriorLights:Electricity [kW](Hourly)',
       'InteriorEquipment:Electricity [kW](Hourly)', 'hour', 'day_of_week',
       'month']
# Inicializar variables para acumular resultados
rmse_list = []
smape_list = []
for df_serie in average_dfs[1]:
    df=df_serie.dropna().copy()
    df["Date/Time"]=pd.to_datetime(df["Date/Time"])
    df.set_index("Date/Time",inplace=True)
    # Extraer características temporales
    df['hour'] = df.index.hour
    df['day_of_week'] = df.index.dayofweek
    df['month'] = df.index.month
    
    X = df.loc[:, columnas].copy()
    y = df.loc[:, 'Electricity:Facility [kW](Hourly)'].copy()
    
    # Escalar las características
    X_scaled = scaler_average_cluster_1.transform(X)
    
    # Predecir y calcular métricas de error
    y_pred = modelo_average_cluster_1.predict(X_scaled)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    smape_value = smape(y, y_pred)  # Calcular sMAPE
    
    # Almacenar resultados
    rmse_list.append(rmse)
    smape_list.append(smape_value)

# Calcular promedios de las métricas de error
average_rmse = np.mean(rmse_list)
average_smape = np.mean(smape_list)

# Resultados
results = {
    "Metricas del modelo": {
        'Average RMSE': average_rmse,
        'Average sMAPE (%)': average_smape
    }
}

In [49]:
# Mostrar los resultados
print("Resultados del modelo")
optimized_rf_results_df=pd.DataFrame(results)
optimized_rf_results_df

Resultados del modelo


,Metricas del modelo
Average RMSE,26.765813
Average sMAPE (%),2.206081


In [112]:
rmse_list

[27.026703412757403,
 27.727829692903583,
 30.12006124291278,
 24.12867249186021,
 24.825796674492913]

# PRUEBAS CLUSTER 2

## Closest

In [56]:
# Lista de columnas para análisis
columnas = ['Fans:Electricity [kW](Hourly)', 'Cooling:Electricity [kW](Hourly)',
       'InteriorLights:Electricity [kW](Hourly)',
       'InteriorEquipment:Electricity [kW](Hourly)', 'hour', 'day_of_week',
       'month']
# Inicializar variables para acumular resultados
rmse_list = []
smape_list = []
for df_serie in closest_dfs[2]:
    df=df_serie.copy()
    df["Date/Time"]=pd.to_datetime(df["Date/Time"])
    df.set_index("Date/Time",inplace=True)
    # Extraer características temporales
    df['hour'] = df.index.hour
    df['day_of_week'] = df.index.dayofweek
    df['month'] = df.index.month
    
    X = df.loc[:, columnas].copy()
    y = df.loc[:, 'Electricity:Facility [kW](Hourly)'].copy()
    
    # Escalar las características
    X_scaled = scaler_closest_cluster_2.transform(X)
    
    # Predecir y calcular métricas de error
    y_pred = modelo_closest_cluster_2.predict(X_scaled)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    smape_value = smape(y, y_pred)  # Calcular sMAPE
    
    # Almacenar resultados
    rmse_list.append(rmse)
    smape_list.append(smape_value)

# Calcular promedios de las métricas de error
average_rmse = np.mean(rmse_list)
average_smape = np.mean(smape_list)

# Resultados
results = {
    "Metricas del modelo": {
        'Average RMSE': average_rmse,
        'Average sMAPE (%)': average_smape
    }
}


In [57]:
# Mostrar los resultados
print("Resultados del modelo ")
optimized_rf_results_df=pd.DataFrame(results)
optimized_rf_results_df

Resultados del modelo 


,Metricas del modelo
Average RMSE,79.941561
Average sMAPE (%),16.025339


## Average

In [58]:
# Lista de columnas para análisis
columnas = ['Fans:Electricity [kW](Hourly)', 'Cooling:Electricity [kW](Hourly)',
       'InteriorLights:Electricity [kW](Hourly)',
       'InteriorEquipment:Electricity [kW](Hourly)', 'hour', 'day_of_week',
       'month']
# Inicializar variables para acumular resultados
rmse_list = []
smape_list = []
for df_serie in average_dfs[2]:
    df=df_serie.dropna().copy()
    df["Date/Time"]=pd.to_datetime(df["Date/Time"])
    df.set_index("Date/Time",inplace=True)
    # Extraer características temporales
    df['hour'] = df.index.hour
    df['day_of_week'] = df.index.dayofweek
    df['month'] = df.index.month
    
    X = df.loc[:, columnas].copy()
    y = df.loc[:, 'Electricity:Facility [kW](Hourly)'].copy()
    
    # Escalar las características
    X_scaled = scaler_average_cluster_2.transform(X)
    
    # Predecir y calcular métricas de error
    y_pred = modelo_average_cluster_2.predict(X_scaled)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    smape_value = smape(y, y_pred)  # Calcular sMAPE
    
    # Almacenar resultados
    rmse_list.append(rmse)
    smape_list.append(smape_value)

# Calcular promedios de las métricas de error
average_rmse = np.mean(rmse_list)
average_smape = np.mean(smape_list)

# Resultados
results = {
    "Metricas del modelo": {
        'Average RMSE': average_rmse,
        'Average sMAPE (%)': average_smape
    }
}


In [59]:
# Mostrar los resultados
print("Resultados del modelo")
optimized_rf_results_df=pd.DataFrame(results)
optimized_rf_results_df

Resultados del modelo


,Metricas del modelo
Average RMSE,23.392239
Average sMAPE (%),1.560823


In [117]:
rmse_list

[24.759659017705108,
 19.029786268807403,
 21.63894027294141,
 26.10129572973282,
 25.431515620352677]

# PRUEBAS CLUSTER 3

## Closest

In [60]:
# Lista de columnas para análisis
columnas = ['Fans:Electricity [kW](Hourly)', 'Cooling:Electricity [kW](Hourly)',
       'InteriorLights:Electricity [kW](Hourly)',
       'InteriorEquipment:Electricity [kW](Hourly)', 'hour', 'day_of_week',
       'month']
# Inicializar variables para acumular resultados
rmse_list = []
smape_list = []
for df_serie in closest_dfs[3]:
    df=df_serie.dropna().copy()
    df["Date/Time"]=pd.to_datetime(df["Date/Time"])
    df.set_index("Date/Time",inplace=True)
    # Extraer características temporales
    df['hour'] = df.index.hour
    df['day_of_week'] = df.index.dayofweek
    df['month'] = df.index.month
    
    X = df.loc[:, columnas].copy()
    y = df.loc[:, 'Electricity:Facility [kW](Hourly)'].copy()
    
    # Escalar las características
    X_scaled = scaler_closest_cluster_3.transform(X)
    
    # Predecir y calcular métricas de error
    y_pred = modelo_closest_cluster_3.predict(X_scaled)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    smape_value = smape(y, y_pred)  # Calcular sMAPE
    
    # Almacenar resultados
    rmse_list.append(rmse)
    smape_list.append(smape_value)

# Calcular promedios de las métricas de error
average_rmse = np.mean(rmse_list)
average_smape = np.mean(smape_list)

# Resultados
results = {
    "Metricas del modelo": {
        'Average RMSE': average_rmse,
        'Average sMAPE (%)': average_smape
    }
}


In [61]:
# Mostrar los resultados
print("Resultados del modelo")
optimized_rf_results_df=pd.DataFrame(results)
optimized_rf_results_df

Resultados del modelo


,Metricas del modelo
Average RMSE,45.097759
Average sMAPE (%),41.222661


In [62]:
rmse_list

[9.066959835535027,
 10.46565449313518,
 68.45180454280451,
 68.74370114605156,
 68.76067719962609]

## Average 

In [63]:
# Lista de columnas para análisis
columnas = ['Fans:Electricity [kW](Hourly)', 'Cooling:Electricity [kW](Hourly)',
       'InteriorLights:Electricity [kW](Hourly)',
       'InteriorEquipment:Electricity [kW](Hourly)', 'hour', 'day_of_week',
       'month']
# Inicializar variables para acumular resultados
rmse_list = []
smape_list = []
for df_serie in average_dfs[3]:
    df=df_serie.dropna().copy()
    df["Date/Time"]=pd.to_datetime(df["Date/Time"])
    df.set_index("Date/Time",inplace=True)
    # Extraer características temporales
    df['hour'] = df.index.hour
    df['day_of_week'] = df.index.dayofweek
    df['month'] = df.index.month
    
    X = df.loc[:, columnas].copy()
    y = df.loc[:, 'Electricity:Facility [kW](Hourly)'].copy()
    
    # Escalar las características
    X_scaled = scaler_average_cluster_3.transform(X)
    
    # Predecir y calcular métricas de error
    y_pred = modelo_average_cluster_3.predict(X_scaled)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    smape_value = smape(y, y_pred)  # Calcular sMAPE
    
    # Almacenar resultados
    rmse_list.append(rmse)
    smape_list.append(smape_value)

# Calcular promedios de las métricas de error
average_rmse = np.mean(rmse_list)
average_smape = np.mean(smape_list)

# Resultados
results = {
    "Metricas del modelo": {
        'Average RMSE': average_rmse,
        'Average sMAPE (%)': average_smape
    }
}


In [64]:
# Mostrar los resultados
print("Resultados del modelo")
optimized_rf_results_df=pd.DataFrame(results)
optimized_rf_results_df

Resultados del modelo


,Metricas del modelo
Average RMSE,7.019017
Average sMAPE (%),2.421630


In [28]:
rmse_list

[6.950168961681085,
 7.092564274942147,
 7.749185976478021,
 6.880926274093964,
 6.422238170246827]